# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('../output_data/city_data.csv')
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,arraial do cabo,-22.9661,-42.0278,20.10,56.0,56.0,11.53,BR,1.660196e+09
1,owen sound,44.5672,-80.9435,18.98,81.0,45.0,1.79,CA,1.660196e+09
2,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hermanus,-34.4187,19.2345,8.78,89.0,98.0,2.95,ZA,1.660196e+09
4,mataura,-46.1927,168.8643,6.93,71.0,31.0,1.84,NZ,1.660196e+09
...,...,...,...,...,...,...,...,...,...
578,antofagasta,-23.6500,-70.4000,13.60,83.0,13.0,0.86,CL,1.660196e+09
579,sao joao da barra,-21.6403,-41.0511,21.13,45.0,88.0,9.44,BR,1.660196e+09
580,mizpe ramon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,ahuimanu,21.4447,-157.8378,27.84,72.0,20.0,8.23,US,1.660196e+09


In [3]:
#drop na rows
city_data_df.dropna(inplace = True) 
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,arraial do cabo,-22.9661,-42.0278,20.10,56.0,56.0,11.53,BR,1.660196e+09
1,owen sound,44.5672,-80.9435,18.98,81.0,45.0,1.79,CA,1.660196e+09
3,hermanus,-34.4187,19.2345,8.78,89.0,98.0,2.95,ZA,1.660196e+09
4,mataura,-46.1927,168.8643,6.93,71.0,31.0,1.84,NZ,1.660196e+09
5,westport,41.1415,-73.3579,24.90,92.0,100.0,1.54,US,1.660196e+09
...,...,...,...,...,...,...,...,...,...
575,okato,-39.2000,173.8833,11.21,52.0,13.0,8.05,NZ,1.660196e+09
577,merauke,-8.4667,140.3333,27.91,71.0,81.0,6.04,ID,1.660196e+09
578,antofagasta,-23.6500,-70.4000,13.60,83.0,13.0,0.86,CL,1.660196e+09
579,sao joao da barra,-21.6403,-41.0511,21.13,45.0,88.0,9.44,BR,1.660196e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Store latitude and longitude in locations and convert to float
locations = city_data_df[["Lat", "Lng"]].astype(float)

# get humidity convert to float
humidity = city_data_df["Humidity"].astype(float)

In [17]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
hotel_df = (city_data_df.loc[(city_data_df['Cloudiness'] == 0) 
            & (city_data_df['Humidity'] < 60) 
            & (city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 30) 
            & (city_data_df['Wind Speed'] < 30)])
hotel_df = hotel_df.dropna()
hotel_df.reset_index(inplace=True)
hotel_df.drop('index', axis=1, inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint george,37.1041,-113.5841,29.03,49.0,0.0,3.60,US,1.660196e+09
1,nizhnevartovsk,60.9344,76.5531,21.01,40.0,0.0,2.00,RU,1.660196e+09
2,kuna,43.4918,-116.4201,26.52,58.0,0.0,3.09,US,1.660196e+09
3,nalut,30.3333,10.8500,24.97,51.0,0.0,4.70,LY,1.660196e+09
4,pecos,31.4229,-103.4932,28.46,47.0,0.0,4.63,US,1.660196e+09
5,carnarvon,-24.8667,113.6333,21.00,27.0,0.0,6.11,AU,1.660196e+09
6,oroville,39.5138,-121.5564,27.11,31.0,0.0,2.57,US,1.660196e+09
7,nanakuli,21.3906,-158.1547,27.97,57.0,0.0,4.12,US,1.660196e+09
8,karratha,-20.7377,116.8463,25.05,17.0,0.0,8.48,AU,1.660196e+09
9,balkhash,46.8440,74.9804,24.34,43.0,0.0,5.00,KZ,1.660196e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df['Hotel Name'] = ""

In [9]:
#store the hotels list
hotels = []

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

# set up a parameters dictionary
    params = {
     "location": f'{lat},{lng}',   
     "radius": 5000,
     "type": 'lodging',
     "key": g_key
     }

    #  Build query URL and request your results
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_json = response.json()
    
    try:
        #append the hotel list
        hotels.append(hotel_json['results'][0]['name'])
    
    except:
        hotels.append("No nearby hotel")

hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint george,37.1041,-113.5841,29.03,49.0,0.0,3.60,US,1.660196e+09,Best Western Plus Abbey Inn
1,nizhnevartovsk,60.9344,76.5531,21.01,40.0,0.0,2.00,RU,1.660196e+09,Luch
2,kuna,43.4918,-116.4201,26.52,58.0,0.0,3.09,US,1.660196e+09,Winfield Springs
3,nalut,30.3333,10.8500,24.97,51.0,0.0,4.70,LY,1.660196e+09,No nearby hotel
4,pecos,31.4229,-103.4932,28.46,47.0,0.0,4.63,US,1.660196e+09,Knights Inn & Suites Pecos Northeast
5,carnarvon,-24.8667,113.6333,21.00,27.0,0.0,6.11,AU,1.660196e+09,Hospitality Carnarvon
6,oroville,39.5138,-121.5564,27.11,31.0,0.0,2.57,US,1.660196e+09,"Holiday Inn Express & Suites Oroville Lake, an..."
7,nanakuli,21.3906,-158.1547,27.97,57.0,0.0,4.12,US,1.660196e+09,Camp Pālehua
8,karratha,-20.7377,116.8463,25.05,17.0,0.0,8.48,AU,1.660196e+09,ibis Styles Karratha
9,balkhash,46.8440,74.9804,24.34,43.0,0.0,5.00,KZ,1.660196e+09,Family


In [10]:
# Store latitude and longitude in locations and convert to float
locationsh = hotel_df[["Lat", "Lng"]].astype(float)

# get humidity convert to float
humidityh = hotel_df["Humidity"].astype(float)

In [11]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locationsh, weights=humidityh, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
# Create heat layer
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Add layer
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))